In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data_tr = pd.read_csv('train.csv')
data_ts = pd.read_csv('test.csv')
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 10
y_tr = data_tr.iloc[:, 1]
x_tr = data_tr.drop(columns='Survived')

In [ ]:
x_tr.head(10)

In [ ]:
x_tr.describe()

In [ ]:
data_tr.boxplot(column='Age', by='Survived')

In [ ]:
plt.figure(figsize=(30, 15))
sns.heatmap(x_tr.isnull(), cbar=False)

In [ ]:
x_tr.isnull().sum()

In [ ]:
miss_thr = 0.25 * len(data_tr)
x_tr = x_tr.dropna(thresh=miss_thr, axis=1)

In [ ]:
x_tr = x_tr.drop(columns=['Name', 'Ticket', 'PassengerId'])
x_tr.head(10)

In [ ]:
x_tr.hist(figsize=(10,10))

In [ ]:
y_tr.hist(figsize=(3,3))

In [ ]:
sns.heatmap(abs(x_tr.corr()))

In [ ]:
# x_tr.replace('NA', np.NaN)
# week = {'Mon': 'Monday', 'Tue': 'Tuesday', 'Wed': 'Wednesday', 'Thur': 'Thursday', 'Fri': 'Friday', 'Sat': 'Saturday', 'Sun': 'Sunday'}
# mapping_dic = {'x3': week}
# cat_cols = cat_cols.replace(mapping_dic)
# cat_cols['x7'] = cat_cols['x7'].str.rstrip('%').astype('float')

In [ ]:
le = LabelEncoder()
x_tr['Sex'] = le.fit_transform(x_tr['Sex'])
x_tr.head(10)

In [ ]:
# cat_cols = x_tr.select_dtypes(include='O')
# num_cols = x_tr.select_dtypes(exclude='O')

imputer = SimpleImputer(strategy='most_frequent')
embarked = pd.DataFrame(x_tr['Embarked'])
imputer.fit(embarked)
x_tr['Embarked'] = pd.DataFrame(imputer.transform(pd.DataFrame(x_tr['Embarked'])), columns=['Embarked'])

# from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
# le = LabelEncoder()
# embarked = pd.DataFrame(le.fit_transform(embarked), columns=['Embarked'])
# rfc = RandomForestClassifier(n_estimators=100, random_state=42)
# rfc.fit(embarked, y_tr)
# x_tr[x_tr['Embarked'].isnull()].drop('Embarked', axis=1)
# x_tr['Embarked'][x_tr['Embarked'].isnull()] = rfc.predict(x_tr[x_tr['Embarked'].isnull()].drop('Embarked', axis=1))

In [ ]:
ohe = OneHotEncoder()
ohencoded = pd.DataFrame(ohe.fit_transform(pd.DataFrame(x_tr['Embarked'])).toarray(), columns=['S', 'C', 'Q'])
x_tr = x_tr.drop(columns=['Embarked'])
x_tr = pd.concat([x_tr, ohencoded], axis = 1)
x_tr

In [ ]:
x_tr.isnull().sum()

In [ ]:
cols = x_tr.columns
knn_imp = KNNImputer(n_neighbors=3)
x_tr = pd.DataFrame(knn_imp.fit_transform(x_tr), columns=cols).astype('float')
x_tr

In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(x_tr), columns = cols)
X_scaled

In [ ]:
threshold = 0.1
selector = VarianceThreshold(threshold)
selector.fit(X_scaled)
selected_indices = selector.get_support(indices=True)
selected_indices
X = X_scaled
# X = X_scaled.drop(columns=['C'])

In [ ]:
k = 6
selector = SelectKBest(score_func=mutual_info_regression, k=k)
x_best = pd.DataFrame(selector.fit_transform(X, y_tr))
selected_features = [i for i, x in enumerate(selector.get_support()) if x]
print(f"Selected Features: {selected_features}")
x_best

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x_best, y_tr, test_size=0.2, random_state=42)

In [ ]:
LR = RFECV(LogisticRegression(max_iter=2000), scoring = 'roc_auc', n_jobs = -1, cv = 5, step = 5)
LR.fit(X_train, y_train)
LR_probs = LR.predict_proba(X_val)
print('AUC: ', roc_auc_score(y_val, LR_probs[:,1]))
print('Accuracy: ', LR.score(X_val, y_val))
y_pred = LR.predict(X_val)
print(classification_report(y_val, y_pred))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 1, 10]}
clf = SVC()
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Validation Score: {grid_search.best_score_}")

In [ ]:
clf = SVC(C=10, kernel='rbf', gamma=0.1)
clf.fit(X_train, y_train)
y_pred2 = clf.predict(X_val)
cm = confusion_matrix(y_val, y_pred2)
sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 1, 10]}
clf = SVC()
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Validation Score: {grid_search.best_score_}")
clf = SVC(C=1, kernel='rbf')
y_pred2 = clf.predict(X_val)
cm = confusion_matrix(y_val, y_pred2)
sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
xgb_model = XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred3 = xgb_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred3)
print('Accuracy:', accuracy)
cm = confusion_matrix(y_val, y_pred3)
sns.heatmap(cm, annot=True, fmt='d')